# SurrealDBVectorStore

>[SurrealDB](https://surrealdb.com/) is an end-to-end cloud-native database designed for modern applications, including web, mobile, serverless, Jamstack, backend, and traditional applications. With SurrealDB, you can simplify your database and API infrastructure, reduce development time, and build secure, performant apps quickly and cost-effectively.
>
>**Key features of SurrealDB include:**
>
>* **Reduces development time:** SurrealDB simplifies your database and API stack by removing the need for most server-side components, allowing you to build secure, performant apps faster and cheaper.
>* **Real-time collaborative API backend service:** SurrealDB functions as both a database and an API backend service, enabling real-time collaboration.
>* **Support for multiple querying languages:** SurrealDB supports SQL querying from client devices, GraphQL, ACID transactions, WebSocket connections, structured and unstructured data, graph querying, full-text indexing, and geospatial querying.
>* **Granular access control:** SurrealDB provides row-level permissions-based access control, giving you the ability to manage data access with precision.
>
>View the [features](https://surrealdb.com/features), the latest [releases](https://surrealdb.com/releases), and [documentation](https://surrealdb.com/docs).

This notebook covers how to get started with the SurrealDB vector store.

## Setup

You can run SurrealDB locally or start with a [free SurrealDB cloud account](https://surrealdb.com/docs/cloud/getting-started).

For local, two options:
1. [Install SurrealDB](https://surrealdb.com/docs/surrealdb/installation) and [run SurrealDB](https://surrealdb.com/docs/surrealdb/installation/running). Run in-memory with:

    ```bash
    surreal start -u root -p root
    ```

2. [Run with Docker](https://surrealdb.com/docs/surrealdb/installation/running/docker).

    ```bash
    docker run --rm --pull always -p 8000:8000 surrealdb/surrealdb:latest start
    ```

## Install dependencies

Install `langchain-surrealdb` and `surrealdb` python packages.

```shell
# -- Using pip
pip install --upgrade langchain-surrealdb surrealdb
# -- Using poetry
poetry add langchain-surrealdb surrealdb
# -- Using uv
uv add --upgrade langchain-surrealdb surrealdb
```

To run this notebook, we just need to install the additional dependencies required by this example:


In [1]:
!poetry add --quiet --group docs langchain-ollama git+https://github.com/surrealdb/langchain-surrealdb.git

## Initialization


In [2]:
from langchain_ollama import OllamaEmbeddings
from langchain_surrealdb.vectorstores import SurrealDBVectorStore
from surrealdb import Surreal

conn = Surreal("ws://localhost:8000/rpc")
conn.signin({"username": "root", "password": "root"})
conn.use("langchain", "demo")
vector_store = SurrealDBVectorStore(OllamaEmbeddings(model="llama3.2"), conn)

## Manage vector store

### Add items to vector store


In [3]:
from langchain_core.documents import Document

_url = "https://surrealdb.com"
d1 = Document(page_content="foo", metadata={"source": _url})
d2 = Document(page_content="SurrealDB", metadata={"source": _url})
d3 = Document(page_content="bar", metadata={"source": _url})
d4 = Document(page_content="this is surreal", metadata={"source": _url})

vector_store.add_documents(documents=[d1, d2, d3, d4], ids=["1", "2", "3", "4"])

['1', '2', '3', '4']

### Update items in vector store

In [4]:
updated_document = Document(
    page_content="zar", metadata={"source": "https://example.com"}
)

vector_store.add_documents(documents=[updated_document], ids=["3"])

['3']

### Delete items from vector store


In [5]:
vector_store.delete(ids=["3"])

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent. 

### Query directly

Performing a simple similarity search can be done as follows:


In [6]:
results = vector_store.similarity_search(
    query="surreal", k=1, custom_filter={"source": "https://surrealdb.com"}
)
for doc in results:
    print(f"{doc.page_content} [{doc.metadata}]")  # noqa: T201

this is surreal [{'source': 'https://surrealdb.com'}]


If you want to execute a similarity search and receive the corresponding scores you can run:


In [7]:
results = vector_store.similarity_search_with_score(
    query="thud", k=1, custom_filter={"source": "https://surrealdb.com"}
)
for doc, score in results:
    print(f"[similarity={score:.0%}] {doc.page_content}")  # noqa: T201

[similarity=57%] this is surreal


### Query by turning into retriever

You can also transform the vector store into a retriever for easier usage in your chains. 


In [8]:
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 1, "lambda_mult": 0.5}
)
retriever.invoke("surreal")

[Document(id='4', metadata={'source': 'https://surrealdb.com'}, page_content='this is surreal')]

## Usage for retrieval-augmented generation

For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:

- [How-to: Question and answer with RAG](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [Retrieval conceptual docs](https://python.langchain.com/docs/concepts/retrieval/)

## API reference

For detailed documentation of all SurrealDBVectorStore features and configurations head to the API reference: https://python.langchain.com/api_reference/surrealdb/index.html

## Next steps

- look at the [basic example](https://github.com/surrealdb/langchain-surrealdb/tree/main/examples/basic). Use the Dockerfile to try it out!
- look at the [graph example](https://github.com/surrealdb/langchain-surrealdb/tree/main/examples/graph)
- [Awesome SurrealDB](https://github.com/surrealdb/awesome-surreal), a curated list of SurrealDB resources, tools, utilities, and applications
